In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from PIL import Image

In [2]:
extract = cv2.imread('images/extract.jpg')

In [3]:
riverPoints = []
nonriverPoints = []

for i in range(extract.shape[0]):
    for j in range(extract.shape[1]):
        if extract[i][j][0] >=240 and extract[i][j][1] >=240 and extract[i][j][2] >=240:
            riverPoints.append([i,j])
        elif extract[i][j][0] <=3 and extract[i][j][1] <=3 and extract[i][j][2] <= 3:
            nonriverPoints.append([i,j])
    
print(len(riverPoints))
print(len(nonriverPoints))

1276
6352


In [4]:
def get_data(path):
    img = Image.open(path, 'r')
    p = np.array(list(img.getdata())).reshape(512,512)
    return p

pr = get_data('images/1.gif')
pg = get_data('images/2.gif')
pb = get_data('images/3.gif')
pi = get_data('images/4.gif')

In [5]:
riverValues = []
nonriverValues = []

for i in range(50):  # get 50 river point data
    x, y = riverPoints[i]
    riverValues.append([pr[x][y], pg[x][y], pb[x][y], pi[x][y]])

for i in range(100):  # get 100 non-river point data
    x, y = nonriverPoints[i]
    nonriverValues.append([pr[x][y], pg[x][y], pb[x][y], pi[x][y]])

In [6]:
riverValues = np.array(riverValues)
print(riverValues.shape)
nonriverValues = np.array(nonriverValues)
print(nonriverValues.shape)

(50, 4)
(100, 4)


In [7]:
# Calculate Mean
riverMean = np.mean(riverValues, axis = 0)
nonriverMean = np.mean(nonriverValues, axis = 0)

In [8]:
# Calculate covariance for river and non river class
diffRiver = riverValues - riverMean
diffNonriver = nonriverValues - nonriverMean

covRiver = (float(1) / float(riverValues.shape[0])) * np.dot(diffRiver.T, diffRiver)
covNonriver = (float(1) / float(nonriverValues.shape[0])) * np.dot(diffNonriver.T, diffNonriver)

#### Creating Test Data

In [9]:
data = np.empty(shape=(512,512, 4),dtype='uint8')
for i in range(512):
    for j in range(512):
        data[i][j][0] = pr[i][j]
        data[i][j][1] = pg[i][j]
        data[i][j][2] = pb[i][j]
        data[i][j][3] = pi[i][j]

In [10]:
def main(P1,P2):
    final_ans = np.zeros((512, 512))
    
    for i in range(512):
        for j in range(512):
            modifiedData = data[i][j] - riverMean
        
            river_class = np.dot(np.dot(modifiedData.T, np.linalg.inv(covRiver)), modifiedData)
            river_prediction = (float(1) / ((float(2) * np.pi) ** 2)) * (float(1) / (np.sqrt(np.linalg.det(covRiver)))) * np.exp((float(-0.5)) * river_class)
        
            modifiedData = data[i][j] - nonriverMean
            non_river_class = np.dot(np.dot(np.transpose((modifiedData)), np.linalg.inv(covNonriver)), modifiedData)
            non_river_prediction =  (float(1) / ((float(2) * np.pi) ** 2)) * (float(1) / (np.sqrt(np.linalg.det(covNonriver)))) * np.exp((float(-0.5)) * non_river_class)
        
            if(P1 * river_prediction >= P2 * non_river_prediction) :
                final_ans[i][j] = 255
            else :
                final_ans[i][j] = 0
    return final_ans

In [11]:
im = Image.fromarray(main(0.7,0.3))
im.show()

In [12]:
im = Image.fromarray(main(0.5,0.5))
im.show()

In [13]:
im = Image.fromarray(main(0.3,0.7))
im.show()